# Project GDP
## GitHub Repo: https://github.com/JackBeerman/GDP

In [1]:
import pandas as pd
import numpy as np
import requests
import os
import zipfile
import io
POSTGRES_PASSWORD = os.getenv('POSTGRES_PASSWORD')
import psycopg2
from sqlalchemy import create_engine

In [2]:
### Goal: Have a dashboard that allows us to select a country, country it trades with, view imports and exports, tarrifs
### Goal: predictive analysis of tarrifs and GDP..? too much

## Data

* The World Bank https://data.worldbank.org/
* World Integrated Trade Solution https://wits.worldbank.org/
* World Trade Organization https://www.wto.org/english/res_e/statis_e/statis_e.htm
* Our World in Data https://ourworldindata.org/trade-and-globalization
* International Monetary Fund https://data.imf.org/?sk=9d6028d4-f14a-464c-a2f2-59b2cd424b85
* Human Development Reports https://hdr.undp.org/

## Preprocess World Bank

In [3]:
exports = pd.read_csv("exports/goods_exported.csv", skiprows=4)
exports.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,Unnamed: 67
0,Aruba,ABW,"Goods exports (BoP, current US$)",BX.GSR.MRCH.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,2.545405e+08,3.296600e+08,2.840210e+08,1.346004e+08,1.848016e+08,1.307081e+08,8.733158e+07,1.230030e+08,2.052746e+08,NaN
1,Africa Eastern and Southern,AFE,"Goods exports (BoP, current US$)",BX.GSR.MRCH.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,2.328337e+11,1.807787e+11,1.713935e+11,1.954152e+11,2.161620e+11,2.026909e+11,1.805053e+11,2.585645e+11,2.892419e+11,NaN
2,Afghanistan,AFG,"Goods exports (BoP, current US$)",BX.GSR.MRCH.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,6.417674e+08,5.778598e+08,6.142176e+08,7.839629e+08,8.752427e+08,8.638344e+08,7.767345e+08,NaN,NaN,NaN
3,Africa Western and Central,AFW,"Goods exports (BoP, current US$)",BX.GSR.MRCH.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,1.657213e+11,1.084194e+11,9.644857e+10,1.192904e+11,1.424634e+11,1.496790e+11,1.182803e+11,1.434662e+11,NaN,NaN
4,Angola,AGO,"Goods exports (BoP, current US$)",BX.GSR.MRCH.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,5.916988e+10,3.318113e+10,2.758888e+10,3.461345e+10,4.075777e+10,3.472556e+10,2.093744e+10,3.358149e+10,5.003799e+10,NaN


In [4]:
imports = pd.read_csv("imports/import_goods_services.csv", skiprows=4)
imports.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,Unnamed: 67
0,Aruba,ABW,"Imports of goods, services and primary income ...",BM.GSR.TOTL.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,2.409753e+09,2.306334e+09,2.198294e+09,2.267071e+09,2.547906e+09,2.488068e+09,1.790058e+09,2.108923e+09,2.654992e+09,NaN
1,Africa Eastern and Southern,AFE,"Imports of goods, services and primary income ...",BM.GSR.TOTL.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,3.779555e+11,3.262006e+11,2.912185e+11,3.194726e+11,3.443009e+11,3.309496e+11,2.730562e+11,3.421715e+11,4.095932e+11,NaN
2,Afghanistan,AFG,"Imports of goods, services and primary income ...",BM.GSR.TOTL.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,8.441108e+09,8.569287e+09,7.387765e+09,8.007677e+09,8.086339e+09,7.416724e+09,7.034027e+09,NaN,NaN,NaN
3,Africa Western and Central,AFW,"Imports of goods, services and primary income ...",BM.GSR.TOTL.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,2.260464e+11,1.934235e+11,1.579141e+11,1.740356e+11,2.137816e+11,2.515093e+11,2.062530e+11,2.239684e+11,NaN,NaN
4,Angola,AGO,"Imports of goods, services and primary income ...",BM.GSR.TOTL.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,6.303338e+10,4.404266e+10,3.127150e+10,3.600158e+10,3.416537e+10,3.050823e+10,2.060563e+10,2.498396e+10,3.743725e+10,NaN


In [5]:
gdp = pd.read_csv("gdp/GDP.csv", skiprows=4)
gdp.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,Unnamed: 67
0,Aruba,ABW,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,2.791061e+09,2.963128e+09,2.983799e+09,3.092179e+09,3.276188e+09,3.395794e+09,2.610039e+09,3.126019e+09,NaN,NaN
1,Africa Eastern and Southern,AFE,GDP (current US$),NY.GDP.MKTP.CD,2.112502e+10,2.161623e+10,2.350628e+10,2.804836e+10,2.592067e+10,2.947210e+10,...,1.006526e+12,9.273485e+11,8.851764e+11,1.021043e+12,1.007196e+12,1.000834e+12,9.275933e+11,1.081998e+12,1.169484e+12,NaN
2,Afghanistan,AFG,GDP (current US$),NY.GDP.MKTP.CD,5.377778e+08,5.488889e+08,5.466667e+08,7.511112e+08,8.000000e+08,1.006667e+09,...,2.055058e+10,1.999814e+10,1.801955e+10,1.889635e+10,1.841886e+10,1.890450e+10,2.014345e+10,1.458314e+10,NaN,NaN
3,Africa Western and Central,AFW,GDP (current US$),NY.GDP.MKTP.CD,1.044764e+10,1.117321e+10,1.199053e+10,1.272769e+10,1.389811e+10,1.492979e+10,...,8.943225e+11,7.686447e+11,6.913634e+11,6.848988e+11,7.670257e+11,8.225384e+11,7.864600e+11,8.444597e+11,8.778633e+11,NaN
4,Angola,AGO,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,1.372444e+11,8.721930e+10,4.984049e+10,6.897277e+10,7.779294e+10,6.930911e+10,5.024137e+10,6.568544e+10,1.067136e+11,NaN


In [6]:
tariffs = pd.read_csv("tariffs/tarrifs.csv", skiprows=4)
tariffs.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,Unnamed: 67
0,Aruba,ABW,"Tariff rate, most favored nation, weighted mea...",TM.TAX.MRCH.WM.FN.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,10.02,10.16,10.3,10.15,12.12,NaN,NaN,NaN,NaN
1,Africa Eastern and Southern,AFE,"Tariff rate, most favored nation, weighted mea...",TM.TAX.MRCH.WM.FN.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,AFG,"Tariff rate, most favored nation, weighted mea...",TM.TAX.MRCH.WM.FN.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,5.72,NaN,NaN,NaN,NaN,NaN
3,Africa Western and Central,AFW,"Tariff rate, most favored nation, weighted mea...",TM.TAX.MRCH.WM.FN.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Angola,AGO,"Tariff rate, most favored nation, weighted mea...",TM.TAX.MRCH.WM.FN.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,10.3,9.38,9.38,NaN,7.68,6.52,9.23,NaN,NaN,NaN


## Preprocess Human Development Reports

need to add openpyxl to requirements

In [17]:
hdi = pd.read_excel("HDR21-22_Statistical_Annex_HDI_Table.xlsx")

In [18]:
hdi.head()

,HDI rank,Country,Human Development Index (HDI),Life expectancy at birth,Expected years of schooling,Mean years of schooling,Gross national income (GNI) per capita,GNI per capita rank minus HDI rank,Previous HDI rank
0,1,Switzerland,0.962,83.9872,16.500299,13.85966,66933.00454,5,3
1,2,Norway,0.961,83.2339,18.185200,13.00363,64660.10622,6,1
2,3,Iceland,0.959,82.6782,19.163059,13.76717,55782.04981,11,2
3,4,"Hong Kong, China (SAR)",0.952,85.4734,17.278170,12.22621,62606.84540,6,4
4,5,Australia,0.951,84.5265,21.054590,12.72682,49238.43335,18,5
